In [1]:
!pip install orjson -q
import numpy as np
import pandas as pd
from tqdm import tqdm
import ast, os, re, json, requests, orjson
from glob import glob
from time import sleep
from bs4 import BeautifulSoup
data_folder = ''

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 3.0 MB/s eta 0:00:00


In [9]:
df = pd.read_csv(data_folder+'data/raw_ade_table_v0924.csv')
df['ade'] = df['txt'].apply(lambda x: str(x).replace('- ', ''))
df.head(1)

,drug,col,txt,ade
0,nexavar,Unnamed: 0,Hepatobi- liary disorders hepatitis* Skin and ...,Hepatobiliary disorders hepatitis* Skin and su...


In [13]:
freqs = ['very common', 'common', 'uncommon', 'rare', 'very rare', 'not known']
socs =  ['blood and lymphatic system disorders','cardiac disorders', 'congenital, familial and genetic disorders',
         'ear and labyrinth disorders', 'endocrine disorders', 'eye disorders', 'gastrointestinal disorders',
         'general disorders and administration site conditions', 'hepatobiliary disorders', 'immune system disorders',
         'infections and infestations', 'injury, poisoning and procedural complications', 'investigations', 'metabolism and nutrition disorders',
         'musculoskeletal and connective tissue disorders', 'neoplasms benign, malignant and unspecified (incl cysts and polyps)',
         'nervous system disorders', 'pregnancy, puerperium and perinatal conditions', 'psychiatric disorders',
         'renal and urinary disorders', 'reproductive system and breast disorders', 'respiratory, thoracic and mediastinal disorders',
         'skin and subcutaneous tissue disorders', 'social circumstances', 'surgical and medical procedures', 'vascular disorders', 'product issues']
titles = ['system organ class', 'frequency', 'adverse events']

In [14]:
f = '' #external folder
meddra_df = pd.read_csv(f+'external_data/umls_meddra_en.csv')
meddra_df['STR'] = meddra_df.STR.apply(lambda x: x.lower())
meddra_df['len'] = meddra_df.STR.apply(lambda x: len(x))
meddra_dict = dict(zip(meddra_df.STR, meddra_df.SDUI))
meddra_df = meddra_df[(meddra_df.TTY == 'PT')|(meddra_df['len'] > 5)]
meddra_df.head(1)

,CUI,LAT,TS,LUI,STT,SUI,ISPREF,AUI,SAUI,SCUI,SDUI,SAB,TTY,CODE,STR,SRL,SUPPRESS,CVF,len
0,C0000727,ENG,P,L0000727,VCW,S0584932,N,A0639292,NaN,NaN,10000647,MDR,PT,10000647,acute abdomen,3,N,256.0,13


In [21]:
found_ades = []
meddra_names = meddra_df.STR.tolist()
for ade_text in tqdm(df.ade.tolist()):
  ar_text = ' '.join(ade_text.split()).lower()
  #remove all of the freqs, socs, and titles
  for f in freqs:
    ar_text = ar_text.replace(f, '')
  for s in socs:
    ar_text = ar_text.replace(s, '')
  for t in titles:
    ar_text = ar_text.replace(t, '')
  found_terms = []
  for concept_name in meddra_names:
    if ar_text.find(concept_name) == -1:
      continue
    else:
      found_terms.append(concept_name)
  found_ades.append(found_terms)

100%|██████████| 5480/5480 [04:16<00:00, 21.35it/s]


In [29]:
df['exact_match_list'] = found_ades
df['exact_match_list'] = df.exact_match_list.apply(lambda x: list(set(x)))
df['matched_codes'] = df.exact_match_list.apply(lambda x: [meddra_dict[i] for i in x] if str(x) != 'nan' else None)
df.to_csv(data_folder+'data/parsed_ade_tabular.csv', index=False)
df.head(1)

,drug,col,txt,ade,exact_match_list,matched_codes
0,nexavar,Unnamed: 0,Hepatobi- liary disorders hepatitis* Skin and ...,Hepatobiliary disorders hepatitis* Skin and su...,"[skin desquamation, inflammation, mucosal infl...","[10040844, 10061218, 10028116, 10050537, 10054..."
